# BUSINESS UNDERSTANDING

## Overivew

SyriaTel, a leading telecommunications company, has been experiencing customer churn—customers stopping their service or switching to competitors. Customer churn is a critical issue as acquiring new customers is often more expensive than retaining existing ones. Thus, predicting which customers are likely to churn soon and implementing strategies to retain them can significantly impact the company's revenue and growth.

## Business Problem

The primary stakeholder for this project is the Chief Marketing Officer (CMO) of SyriaTel. The CMO is responsible for overseeing the company's marketing strategies and customer retention efforts. By identifying patterns of customer churn, the CMO can implement targeted marketing campaigns and retention strategies to minimize churn rates and enhance customer loyalty.

## Objectives

1. Explain causes of customer churn rate.
2. Predict customer churn rate.
3. Reduce customer churn rate.

## DATA UNDERSTANDING

For this project the dataset we will be using is the SyriaTel Customer Chrun. This data set includes only 1 csv file

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import statsmodels.api as sm
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import KMeansSMOTE
from sklearn.utils import resample
from imblearn.over_sampling import SMOTENC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from scipy.sparse import vstack
import xgboost as xgb
from datetime import datetime